<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/DM_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install colab-xterm -q
%load_ext colabxterm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 3.6 MB/s eta 0:00:00


In [ ]:
# %xterm
# curl -fsSL https://ollama.com/install.sh | sh
# ollama serve &

In [ ]:
%xterm

In [ ]:
!ollama pull deepseek-r1:1.5b

In [ ]:
!ollama list

NAME                ID              SIZE      MODIFIED      
deepseek-r1:1.5b    a42b25d8c10a    1.1 GB    6 seconds ago    


In [ ]:
!pip install langchain -q
!pip install langchain-core -q
!pip install langchain-community -q

!pip install -U langchain langchain-ollama -q
!pip install geopy -q

In [ ]:
!wget https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat -O airport_data.dat

import json
import random
from langchain_ollama import OllamaLLM
from datetime import datetime, timedelta
from geopy.distance import geodesic

def create_airport_data_json(input_file, output_file):
    """Creates airport_data.json from airports.dat"""
    airports_data = {}
    with open(input_file, 'r', encoding='utf-8') as f:
        for line in f:
            data = line.strip().split(',')
            if len(data) >= 14:  # Ensure enough data fields
                airport_code = data[4].strip('"')
                if airport_code:  # Skip lines without airport code
                    try:
                        airports_data[airport_code] = {
                            'name': data[1].strip('"'),
                            'latitude': float(data[6]),
                            'longitude': float(data[7])
                        }
                    except ValueError:
                        pass  # Ignore lines with invalid data

    with open(output_file, 'w') as outfile:
        json.dump(airports_data, outfile, indent=4)

# Create the JSON file
create_airport_data_json('airport_data.dat', 'airport_data.json')

CREW RESERVES

In [ ]:
import random
import sqlite3

def create_crew_members(role, num_members):
    """Creates crew members with the specified role."""
    crew = []
    for i in range(num_members):
        crew.append((f"{role} {i+1}", role))  # Tuple with name and role
    return crew

# Generate crew members
pilots = create_crew_members("Pilot", 20)  # 20 pilots
first_officers = create_crew_members("First Officer", 20)  # 20 first officers
flight_attendants = create_crew_members("Flight Attendant", 60)  # 60 flight attendants

# Connect to the database (replace with your actual database path)
conn = sqlite3.connect('crew_members.db')
cursor = conn.cursor()

# Create the table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS crew_members (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        role TEXT NOT NULL
    )
''')

# Insert crew members into the database
cursor.executemany("INSERT INTO crew_members (name, role) VALUES (?, ?)", pilots)
cursor.executemany("INSERT INTO crew_members (name, role) VALUES (?, ?)", first_officers)
cursor.executemany("INSERT INTO crew_members (name, role) VALUES (?, ?)", flight_attendants)

# Commit the changes and close the connection
conn.commit()
conn.close()

In [ ]:
class Flight:
    def __init__(self, flight_number, origin, destination, departure_time, seats_available, aircraft_type):
        self.flight_number = flight_number
        self.origin = origin
        self.destination = destination
        self.departure_time = self.format_departure_time(departure_time)
        self.seats_available = seats_available
        self.aircraft_type = aircraft_type

    def format_departure_time(self, departure_time):
        # Add AM/PM to departure time
        time_obj = datetime.strptime(departure_time, "%H:%M")
        return time_obj.strftime("%I:%M %p")

class Passenger:
    def __init__(self, name, origin, destination, email, ticket_number):
        self.name = name
        self.origin = origin
        self.destination = destination
        self.email = email
        self.ticket_number = ticket_number

class Crew:
    def __init__(self, name, role, location, duty_hours, is_reserve=False):
        self.name = name
        self.role = role
        self.location = location
        self.duty_hours = duty_hours
        self.is_reserve = is_reserve  # Add is_reserve attribute

class Aircraft:
    def __init__(self, aircraft_id, location, status, aircraft_type, is_reserve=False):
        self.aircraft_id = aircraft_id
        self.location = location
        self.status = status
        self.aircraft_type = aircraft_type
        self.is_reserve = is_reserve  # Add is_reserve attribute

class AI_Agent:
    def __init__(self, flights, crew, aircraft, airports_data, crew_reserves):
        self.flights = flights
        self.crew = crew
        self.aircraft = aircraft
        self.airports_data = airports_data
        self.crew_reserves = crew_reserves

    def find_alternative_flight(self, passenger, cancelled_flight):
        print(f"Finding alternative for flight: {cancelled_flight.flight_number} from {cancelled_flight.origin} to {cancelled_flight.destination}")
        alternative_flights = [
            flight for flight in self.flights
            if flight.origin == passenger.origin and
               flight.destination == passenger.destination and
               flight.seats_available > 0 and
               flight != cancelled_flight
        ]
        print(f"Found {len(alternative_flights)} potential alternatives.")
        if alternative_flights:
            return random.choice(alternative_flights)
        else:
            print("No alternative flights found.")
            return None

    def find_nearby_airports(self, airports_data, origin, radius_km):
        """
        Finds nearby airports within a given radius.
        """
        origin_coords = (airports_data[origin]['latitude'], airports_data[origin]['longitude'])
        nearby_airports = []
        for airport, data in airports_data.items():
            if airport != origin:
                airport_coords = (data['latitude'], data['longitude'])
                distance = geodesic(origin_coords, airport_coords).km
                if distance <= radius_km:
                    nearby_airports.append(airport)
        return nearby_airports

    def find_available_crew(self, flight, role):
        """
        Finds available crew members for a given flight and role,
        considering nearby airports and crew reserves.
        """
        # Find regular crew at the origin airport
        available_crew = [
            crew_member for crew_member in self.crew
            if crew_member.role == role and
               crew_member.location == flight.origin and
               crew_member.duty_hours < 10 and
               not crew_member.is_reserve
        ]
        if available_crew:
            # Sort by duty hours to prioritize the most rested crew members
            available_crew.sort(key=lambda x: x.duty_hours)
            return available_crew[0]

        # If no regular crew at origin, find regular crew at nearby airports
        nearby_airports = self.find_nearby_airports(self.airports_data, flight.origin, 300)
        for airport in nearby_airports:
            available_crew = [
                crew_member for crew_member in self.crew
                if crew_member.role == role and
                   crew_member.location == airport and
                   crew_member.duty_hours < 10 and
                   not crew_member.is_reserve
            ]
            if available_crew:
                available_crew.sort(key=lambda x: x.duty_hours)
                return available_crew[0]

        # No available regular crew at the origin or nearby airports
        # Check for crew reserves
        available_crew = [
            crew_member for crew_member in self.crew_reserves
            if crew_member.role == role and
               crew_member.duty_hours < 10
        ]
        if available_crew:
            # Sort by duty hours to prioritize the most rested crew members
            available_crew.sort(key=lambda x: x.duty_hours)
            return available_crew[0]
        else:
            return None

    def find_available_aircraft(self, flight):
        """
        Finds available aircraft for a given flight,
        considering reserve aircraft if necessary.
        """
        # Find regular aircraft at the origin airport
        alternative_aircraft = [
            ac for ac in self.aircraft
            if ac.location == flight.origin and
               ac.status == "Available" and
               (ac.aircraft_type == flight.aircraft_type or
                self.is_similar_aircraft_type(ac.aircraft_type, flight.aircraft_type)) and
               not ac.is_reserve  # Exclude reserve aircraft for now
        ]
        if alternative_aircraft:
            alternative_aircraft.sort(key=lambda ac: self.get_aircraft_capacity(ac.aircraft_type))
            for ac in alternative_aircraft:
                if self.get_aircraft_capacity(ac.aircraft_type) >= flight.seats_available:
                    return ac
            return None  # No suitable regular aircraft found

        # If no regular aircraft available, check for reserve aircraft
        alternative_aircraft = [
            ac for ac in self.aircraft
            if ac.location == flight.origin and
               ac.status == "Available" and
               (ac.aircraft_type == flight.aircraft_type or
                self.is_similar_aircraft_type(ac.aircraft_type, flight.aircraft_type)) and
               ac.is_reserve
        ]
        if alternative_aircraft:
            alternative_aircraft.sort(key=lambda ac: self.get_aircraft_capacity(ac.aircraft_type))
            for ac in alternative_aircraft:
                if self.get_aircraft_capacity(ac.aircraft_type) >= flight.seats_available:
                    return ac
            return None  # No suitable reserve aircraft found
        else:
            return None  # No aircraft available

    def get_aircraft_capacity(self, aircraft_type):
        capacities = {"A320": 150, "B737": 180, "B787": 300, "B777": 400}
        return capacities.get(aircraft_type, 0)

    def get_nearby_airports(self, airport_code, max_distance=300):
        airport = self.airports_data.get(airport_code)
        if not airport:
            return []

        nearby_airports = []
        for other_airport_code, other_airport in self.airports_data.items():
            if other_airport_code != airport_code:
                distance = geodesic((airport['latitude'], airport['longitude']),
                                   (other_airport['latitude'], other_airport['longitude'])).km
                if distance <= max_distance:
                    nearby_airports.append(other_airport_code)
        return nearby_airports

    def is_similar_aircraft_type(self, type1, type2):
        capacities = self.get_aircraft_capacity
        capacity1 = capacities(type1)
        capacity2 = capacities(type2)
        return abs(capacity1 - capacity2) / max(capacity1, capacity2, 1) <= 0.2

class LLM_Assistant:
    def __init__(self, model_name="deepseek-r1:1.5B", temperature=0.0, max_tokens=2048, n_ctx=4096):
        self.llm = OllamaLLM(
            model=model_name,
            temperature=temperature,
            max_tokens=max_tokens,  # Pass max_tokens directly
            n_ctx=n_ctx            # Pass n_ctx directly
        )

    def generate_response(self, situation, passenger=None, cancelled_flight=None, alternative_flight=None, airline_name="SUN AIRLINES", **kwargs):
        if situation == "cancellation":
            # 1. Generate the apology statement
            apology_prompt = f"""
            You are a helpful AI assistant working for {airline_name}.
            Generate a concise and professional apology for the cancellation of a flight.
            Focus on expressing regret for the inconvenience caused.
            **Example:** "We sincerely apologize for the inconvenience this cancellation may cause."
            **Important:** Do not include emojis or informal language.
            """
            apology_statement = self.llm.invoke(apology_prompt)

            # 2. Generate the cancellation statement
            cancellation_statement_prompt = f"""
            You are a helpful AI assistant working for {airline_name}.
            State that flight {kwargs['flight_number']} from {passenger.origin} to {passenger.destination}
            scheduled for {kwargs['date']} has been cancelled due to {kwargs['reason']}.
            **Example:** "Flight {kwargs['flight_number']} from {passenger.origin} to {passenger.destination},
                        scheduled to depart on {kwargs['date']}, has been cancelled due to {kwargs['reason']}."
            **Important:** Use the provided flight details and reason exactly as given.
            """
            cancellation_statement = self.llm.invoke(cancellation_statement_prompt)

            # 3. Generate the rebooking options section
            rebooking_prompt = f"""
            You are a helpful AI assistant working for {airline_name}.
            Explain to {passenger.name} how to explore rebooking options for a cancelled flight.
            Mention visiting the airline's website or contacting customer support.
            """
            rebooking_response = self.llm.invoke(rebooking_prompt)

            # 4. Generate the refund information section
            refund_prompt = f"""
            You are a helpful AI assistant working for {airline_name}.
            Inform {passenger.name} that they are eligible for a full refund for their cancelled flight.
            Provide instructions on how to request the refund (e.g., through the website or customer support).
            """
            refund_response = self.llm.invoke(refund_prompt)

            # 5. Combine the sections into a complete message
            response = f"""Dear {passenger.name},

            {apology_statement}
            {cancellation_statement}

            {rebooking_response}

            {refund_response}

            Please contact us at [phone number] or [email address] if you have any questions or concerns.

            We appreciate your understanding.

            Sincerely,
            {airline_name}
            (This message was generated by an AI assistant and reviewed by a human agent.)
            """
            return response

        elif situation == "rebooking":
            prompt = f"Inform the passenger, {passenger.name}, that they have been rebooked on flight {kwargs['flight_number']} departing at {kwargs['departure_time']}." # Include passenger name here
            response = self.llm.invoke(prompt)
            return response

        elif situation == "no_alternatives":
            prompt = "We regret to inform you that there are no alternative flights available at this time."
            response = self.llm.invoke(prompt)
            return response
        elif situation == "resource_unavailable":
              prompt = f"""
              Generate a message to inform the passenger, {passenger.name}, that we are unable to assign
              sufficient crew or aircraft for the alternative flight {kwargs['flight_number']}.
              Explain that we are working to resolve the situation and will provide updates soon.
              Offer options for contacting customer support or exploring other travel arrangements.
              """ # Include passenger name here
              response = self.llm.invoke(prompt)
              return response


# Load airport data
try:
    with open('airport_data.json', 'r') as f:
        airports_data = json.load(f)
except FileNotFoundError:
    print("Error: airport_data.json not found. Please provide a valid airport data file.")
    airports_data = {}




# Generate Sample Data
airports = ["JFK", "LAX", "ORD", "ATL", "DFW", "DEN", "SFO", "SEA", "MIA", "CLT"]
asian_airports = ["HND", "ICN", "PVG", "HKG", "SIN"]
european_airports = ["LHR", "CDG", "FRA", "AMS", "MAD"]
caribbean_airports = ["CUN", "PUJ", "MBJ", "SJU", "NAS"]
aircraft_types = ["A320", "B737", "B787", "B777"]
flights = []

def add_time(time_str, minutes_to_add):
    time_obj = datetime.strptime(time_str, "%H:%M")
    new_time_obj = time_obj + timedelta(minutes=minutes_to_add)
    return new_time_obj.strftime("%H:%M")

# Generate domestic flights
for i in range(1000):
    origin = random.choice(airports)
    destination = random.choice([a for a in airports if a != origin])
    if random.random() < 0.3:
        connecting_airport = random.choice([a for a in airports if a != origin and a != destination])
        departure_time = f"{random.randint(6, 23):02d}:{random.randint(0, 59):02d}"
        flights.append(Flight(f"{random.choice(['AA', 'UA', 'DL', 'B6'])}{random.randint(100, 999)}", origin, connecting_airport,
                               departure_time, random.randint(0, 150), random.choice(aircraft_types)))
        next_departure_time = add_time(departure_time, 25)
        flights.append(Flight(f"{random.choice(['AA', 'UA', 'DL', 'B6'])}{random.randint(100, 999)}", connecting_airport, destination,
                               next_departure_time, random.randint(0, 150), random.choice(aircraft_types)))
    else:
        departure_time = f"{random.randint(6, 23):02d}:{random.randint(0, 59):02d}"
        flights.append(Flight(f"{random.choice(['AA', 'UA', 'DL', 'B6'])}{random.randint(100, 999)}", origin, destination,
                               departure_time, random.randint(0, 150), random.choice(aircraft_types)))

# Generate international flights to Asia
for _ in range(5):
    origin = random.choice(airports)
    destination = random.choice(asian_airports)
    departure_time = f"{random.randint(6, 23):02d}:{random.randint(0, 59):02d}"
    flights.append(Flight(f"{random.choice(['AA', 'UA', 'DL', 'B6'])}{random.randint(100, 999)}", origin, destination,
                           departure_time, random.randint(0, 300), random.choice(["B787", "B777"])))

# Generate international flights to Europe
for _ in range(5):
    origin = random.choice(airports)
    destination = random.choice(european_airports)
    departure_time = f"{random.randint(6, 23):02d}:{random.randint(0, 59):02d}"
    flights.append(Flight(f"{random.choice(['AA', 'UA', 'DL', 'B6'])}{random.randint(100, 999)}", origin, destination,
                           departure_time, random.randint(0, 300), random.choice(["B787", "B777"])))

# Generate flights to the Caribbean
for _ in range(5):
    origin = random.choice(airports)
    destination = random.choice(caribbean_airports)
    departure_time = f"{random.randint(6, 23):02d}:{random.randint(0, 59):02d}"
    flights.append(Flight(f"{random.choice(['AA', 'UA', 'DL', 'B6'])}{random.randint(100, 999)}", origin, destination,
                           departure_time, random.randint(0, 150), random.choice(["A320", "B737"])))

# Create crew members with reserves
crew = []
for i in range(100):
    is_reserve = random.random() < 0.3  # 30% chance of being a reserve crew member
    crew.append(Crew(f"Crew Member {i}", random.choice(["Pilot", "Flight Attendant"]),
                     random.choice(airports), random.randint(0, 8), is_reserve))

# Create aircraft with reserves
aircraft = []
for i in range(50):
    is_reserve = random.random() < 0.2  # 20% chance of being a reserve aircraft
    aircraft.append(Aircraft(f"Aircraft {i}", random.choice(airports),
                             random.choice(["Available", "Maintenance", "In Flight"]),
                             random.choice(aircraft_types), is_reserve))

# Sample Passenger with more information
passenger = Passenger("Alice", "JFK", "LAX", "alice@example.com", "1234567890")

# Sample Cancelled Flight
cancelled_flight = flights[10]

import pandas as pd   # Import the library and alias it as 'pd'

# Load crew reserves from the database
conn = sqlite3.connect('crew_members.db')
crew_reserves_df = pd.read_sql_query("SELECT * FROM crew_members", conn)
conn.close()

# Convert DataFrame to a list of Crew objects
crew_reserves = []
for index, row in crew_reserves_df.iterrows():
    crew_reserves.append(Crew(row['name'], row['role'], "Reserve", 0))  # Location set to "Reserve"

# AI Agent with airport data and crew reserves
agent = AI_Agent(flights, crew, aircraft, airports_data, crew_reserves)  # Pass crew_reserves to the constructor


# Initialize LLM Assistant
llm_assistant = LLM_Assistant(model_name="deepseek-r1:1.5b")

# Main Logic
alternative_flight = agent.find_alternative_flight(passenger, cancelled_flight)


print(f"Alternative flight after find_alternative_flight: {alternative_flight}")  # New print statement


if alternative_flight:
    print(llm_assistant.generate_response(
        "cancellation", flight_number=cancelled_flight.flight_number, reason="mechanical issues",
        date=cancelled_flight.departure_time, passenger=passenger
    ))

    print(f"Alternative flight before rebooking response: {alternative_flight}")  # New print statement


    print(llm_assistant.generate_response(
           "rebooking", flight_number=alternative_flight.flight_number, departure_time=alternative_flight.departure_time, passenger=passenger  # Pass passenger here
       ))

    pilot = agent.find_available_crew(alternative_flight, "Pilot")
    flight_attendants = []
    for _ in range(3):
        attendant = agent.find_available_crew(alternative_flight, "Flight Attendant")
        if attendant:
            flight_attendants.append(attendant)
            attendant.duty_hours += 4  # Adding 4 hours for flight duration example
            # Remove attendant to avoid double booking in real scenario
            # agent.crew.remove(attendant)
        else:
            break
    new_aircraft = agent.find_available_aircraft(alternative_flight)
    if pilot and len(flight_attendants) == 3 and new_aircraft:
        print("Crew assigned:")
        print(f"  Pilot: {pilot.name}")
        print(f"  Flight Attendants: {[attendant.name for attendant in flight_attendants]}")
        print(f"Aircraft assigned: {new_aircraft.aircraft_id}")
    else:
        print(llm_assistant.generate_response(
            "resource_unavailable", flight_number=alternative_flight.flight_number, passenger=passenger  # Add passenger here
        ))

else:
    print(llm_assistant.generate_response(
        "cancellation", flight_number=cancelled_flight.flight_number, reason="air traffic control restrictions",
        date=cancelled_flight.departure_time, passenger=passenger
    ))
    print(llm_assistant.generate_response("no_alternatives"))

In [ ]:
#NEW-CODE